In [3]:
class CarInterface:

    def __init__(self):
        self.logs = []

    #log the current state of the car
    def log(self):
        self.logs.append(self.get_range())

    #return logs
    def get_logs(self):
        return self.logs

    #returns make/model of the car
    def get_model(self):
        pass

    #returns range of car with full charge
    def get_max_range(self):
        pass

    #returns current range on the car (miles)
    def get_range(self):
        pass

    #returns current charge percent of car
    def get_charge(self):
        pass

    #returns max charge watts for car
    def get_max_charge(self):
        pass

    #simulate driving miles, adjusting range/charge
    def drive(self, miles):
        pass

    #simulate charging, adjusting range/charge
    def charge(self, duration, power):
        pass

class AgentInterface:

    def __init__(self):
        self.logs = []
    
    #log the current state of the Agent
    def log(self):
        self.logs.append(self.get_location)
    
    #return the logs
    def get_logs(self):
        return self.logs

    #returns agent id
    def get_id(self):
        pass

    #returns agent's nth car
    def get_car(self, n = 0):
        pass

    #return current location
    def get_location(self):
        pass

    #return "drive", "charge", or None
    def get_action(self, time):
        pass

    #return desired charging location if taking "charge" action
    def charge_location(self, time, stations):
        pass

    #return desired location to drive if taking "drive" action
    def drive_location(self, time):
        pass

    #return agent's current location
    def get_location(self, time):
        pass

    #adjust location and car info
    def drive(self, destination, miles):
        pass

    #decide how long to leave the car plugged in
    def plug_duration(self, time, preference):
        if preference == "schedule_charge":
            # Implement logic for scheduling charge based on price drops over a 6 hour time period
            pass
        elif preference == "naive_charge":
            # Charge the car from the time of plug-in to 80%
            return min((0.8 - self.get_car().get_charge()) * self.get_car().get_max_range() / self.get_car().get_max_charge(), time)
        else:
            return 0

    #return boolean if the agent is currently doing another action (like driving or charging)
    def is_busy(self, time):
        pass

    # Check if the car is charged to 80%
    def is_car_charged_to_80(self):
        return self.get_car().get_charge() >= 0.8


In [2]:
class StationInterface:

    def __init__(self):
        self.logs = []

    def log(self, time):
        self.logs.append((self.get_price(time), self.get_queue_length))

    def get_logs(self):
        return self.logs

    # return station id
    def get_id(self):
        pass

    # return station name (maybe not necessary)
    def get_name(self):
        pass

    # return station location (latitude, longitude)
    def get_location(self):
        pass

    # return price to charge at time time
    def get_price(self, time):
        pass

    # return total number of chargers at this location
    def get_num_chargers(self):
        pass

    # return how many chargers are free to use
    def get_num_available(self, time):
        pass

    # return a list of all users currently charging
    def get_current_users(self, time):
        pass

    # return station power supply
    def get_power(self):
        pass

    # Add agent to the queue
    def enqueue_agent(self, agent, time):
        self.queue.append((agent, time))

    # Dequeue agent from the queue
    def dequeue_agent(self, agent):
        for i, (queued_agent, _) in enumerate(self.queue):
            if queued_agent.get_id() == agent.get_id():
                self.queue.pop(i)
                break

    # Get the number of agents in the queue
    def get_queue_length(self):
        return len(self.queue)

    # Get the waiting time for a particular agent
    def get_agent_waiting_time(self, agent):
        waiting_time = 0
        num_chargers = self.get_num_chargers()
        queue_copy = self.queue.copy()

        while queue_copy:
            current_agent, arrival_time = queue_copy.pop(0)
            if current_agent.get_id() == agent.get_id():
                break
            waiting_time += 1
            num_chargers -= 1
            if num_chargers == 0:
                num_chargers = self.get_num_chargers()

        return waiting_time

    # Update the charging status of agents
    def update_charging_agents(self, time):
        charging_agents = self.get_current_users(time)
        for agent in charging_agents:
            if agent.is_car_charged_to_80():
                self.dequeue_agent(agent)


In [1]:
POPULATION = ...
NUM_STATIONS = ...

agents = []
"""
code for generating different agents
"""

stations = []
"""
code for generating different stations
"""
# logs: each timestep has (agent locations, stations)
logs = []

# TODO: logging for analysis
for time in range(24 * 60):
    # Update charging agents at each station
    station_logs = []
    for station in stations:
        station.update_charging_agents(time)
        station_logs.append(station.get_queue_length())
    
    agent_logs = []
    for agent in agents:
        agent_logs.append(agent.get_location())
        if agent.is_busy(time):
            continue
        agent_action = agent.get_action(time)
        if agent_action == "drive":
            destination = agent.drive_location(time)
            agent.drive(destination)
        elif agent_action == "charge":
            station = agent.charge_location(time, stations)
            agent.drive(station)
            
            # Handle station queuing
            station.enqueue_agent(agent, time)
            waiting_time = station.get_agent_waiting_time(agent)

            # Wait for the waiting_time duration before starting the charge
            charge_start_time = time + waiting_time
            charge_duration = agent.plug_duration(charge_start_time)
            agent.get_car().charge(charge_duration, station.get_power())
        else:
            continue
    
    logs.append((agent_logs, station_logs))
